In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## Cargamos el Set de Entrenamiento y el Test Final

In [2]:
train = pd.read_csv('modelo.csv')
test_final = pd.read_csv('test_final.csv')

id_aviso_postulante = test_final['id']

## Realizamos pruebas con el Set de Entrenamiento

Particionamos nuestro set de entrenamiento para testear los resultados de forma local. La idea es realizar varias ejecuciones hasta encontrar los hiper-parámetros que consideramos óptimos, para luego utilizarlos con el test final.

In [11]:
test_s = 0.25
random_s = 0
n_samples = 1000000

no = train['se_postulo'] == 0
si = train['se_postulo'] == 1

train_no = train[no].sample(n_samples)
train_si = train[si].sample(n_samples)

train_final = pd.merge(train_no, train_si, how='outer')

buenos_aires = (train_final['nombre_zona'] == 'Capital Federal') |\
              (train_final['nombre_zona'] == 'Gran Buenos Aires') |\
               (train_final['nombre_zona'] == 'Buenos Aires (fuera de GBA)')

train_final.loc[(buenos_aires), 'Buenos Aires'] = 1
train_final.loc[(~buenos_aires), 'Buenos Aires'] = 0
train_final.loc[(train_final['nombre_zona'].isnull()), 'Buenos Aires'] = 0
train_final.loc[(train_final['edad'].isnull(), 'edad')] = 0

x = np.array(train_final[['edad', 'sexo', 'orden_estudio', 'nivel_laboral', 'tipo_de_trabajo', 'Buenos Aires']])
y = np.array(train_final['se_postulo'])

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_s, random_state=random_s)

In [12]:
# Creamos el Random Forest
random_forest = RandomForestClassifier(n_estimators=25, max_features='sqrt', max_depth=5, min_samples_split=2,\
                                   min_samples_leaf=5, bootstrap=True, oob_score=True, warm_start=True)

# Lo entrenamos con nuestro set de entrenamiento
random_forest.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = random_forest.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

precision

C:\Users\Francisco\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\Francisco\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


0.687492

## Predecimos el Test Final 

En base a los resultados obtenidos con el set de entrenamiento, utilizaremos los hiper-parámetros que consideramos óptimos para predecir las postulaciones del test final.

In [13]:
x_test = np.array(test_final[['edad', 'sexo', 'orden_estudio', 'nivel_laboral', 'tipo_de_trabajo', 'Buenos Aires']])

random_forest = RandomForestClassifier(n_estimators=25, max_features='sqrt', max_depth=5, min_samples_split=2,\
                                   min_samples_leaf=5, bootstrap=False, oob_score=False, warm_start=False)

# Lo entrenamos con la totalidad del set de datos
random_forest.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = random_forest.predict(x_test)

## Submit File

Generamos el Submit File para subir a la competencia de Kaggle

In [14]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})

submit.to_csv('submit_random_forest.csv', index=False)

In [15]:
submit['sepostulo'].value_counts()

1    95075
0     4925
Name: sepostulo, dtype: int64